In [3]:
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

## MCTS Chess Neural Network
---

[notes go here]

In [8]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

class SimpleChessNet():
    
    def __init__(self, action_size, 
                 board_input_shape=(8,8,6), 
                 n_conv_channels=12, 
                 dropout_rate=0.5,
                 learning_rate=1e-3):
        
        self.board_x, self.board_y, self.board_channels = board_input_shape
        self.action_size = action_size
        
        self.input_layer = Input(shape=board_input_shape)
        
        h_conv1 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='same', use_bias=False)(self.input_layer))) # batch_size  x board_x x board_y x num_channels
        h_conv2 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='same', use_bias=False)(h_conv1)))          # batch_size  x board_x x board_y x num_channels
        h_conv3 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='valid', use_bias=False)(h_conv2)))         # batch_size  x (board_x-2) x (board_y-2) x num_channels
        h_conv4 = Activation('elu')(BatchNormalization(axis=3)(Conv2D(n_conv_channels, 3, padding='valid', use_bias=False)(h_conv3)))         # batch_size  x (board_x-4) x (board_y-4) x num_channels
        h_conv4_flat = Flatten()(h_conv4)
        
        s_fc1 = Dropout(dropout_rate)(Activation('elu')(BatchNormalization(axis=1)(Dense(1024, use_bias=False)(h_conv4_flat))))  # batch_size x 1024
        s_fc2 = Dropout(dropout_rate)(Activation('elu')(BatchNormalization(axis=1)(Dense(512, use_bias=False)(s_fc1))))          # batch_size x 1024
        
        # get the policy and value estimates:
        self.pi = Dense(self.action_size, activation='softmax', name='pi')(s_fc2) # batch_size x self.action_size
        self.v = Dense(1, activation='tanh', name='v')(s_fc2)
        
        self.model = Model(inputs=self.input_layer, outputs=[self.pi, self.v])
        self.model.compile(loss=['categorical_crossentropy', 'mean_squared_error'], optimizer=Adam(learning_rate))
        
    def save(self, path):
        tf.saved_model.save(self.model, path)


In [10]:
nn = SimpleChessNet(action_size=64*64)
nn.model.summary()
nn.save('simple_chess_net')

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 8, 8, 6)]    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 8, 8, 12)     648         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 8, 8, 12)     48          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_12 (Activation)      (None, 8, 8, 12)     0           batch_normalization_12[0][0]     
____________________________________________________________________________________________